## Compute Shapley values from orbital resonance effect contributing to d-band center shift of SAAs relative to Pt

### Load python libraries

In [8]:
import pandas as pd
import numpy as np
from ase.io.trajectory import Trajectory
from ase.visualize import view
import matplotlib.pyplot as plt
from pylab import *
from ase import io
from scipy import integrate
from scipy import interpolate
import waterfall_chart
import random
from scipy.stats import norm
import pickle
from ase.db import connect
import shap

### Load data

In [10]:
reference_data = {'Sc': {'d_cen': 1.7032, 'rd': 0.9409409409409409, 'dij': 3.27, 'Wd': 5.999737383}, 
                  'Ti': {'d_cen': 1.3243, 'rd': 0.7907907907907907, 'dij': 2.91, 'Wd': 6.741977763}, 
                   'V': {'d_cen': 0.5548, 'rd': 0.6906906906906907, 'dij': 2.7, 'Wd': 6.977374918}, 
                  'Cr': {'d_cen': -0.0876, 'rd': 0.6306306306306306, 'dij': 2.56, 'Wd': 7.209708128}, 
                  'Mn': {'d_cen': -0.6036, 'rd': 0.5905905905905906, 'dij': 2.48, 'Wd': 7.084501084}, 
                  'Fe': {'d_cen': -0.9278, 'rd': 0.6206206206206206, 'dij': 2.53, 'Wd': 7.112329237}, 
                  'Co': {'d_cen': -1.5905, 'rd': 0.5605605605605606, 'dij': 2.45, 'Wd': 6.696932181}, 
                  'Ni': {'d_cen': -1.6686, 'rd': 0.5205205205205206, 'dij': 2.49, 'Wd': 5.354092732}, 
                  'Cu': {'d_cen': -2.6521, 'rd': 0.4904904904904905, 'dij': 2.58, 'Wd': 4.244656352}, 
                   'Y': {'d_cen': 2.2707, 'rd': 1.2512512512512513, 'dij': 3.58, 'Wd': 7.873690234}, 
                  'Zr': {'d_cen': 1.6485, 'rd': 1.0710710710710711, 'dij': 3.2, 'Wd': 8.905095875}, 
                  'Nb': {'d_cen': 0.6878, 'rd': 0.9409409409409409, 'dij': 2.98, 'Wd': 8.976212519}, 
                  'Mo': {'d_cen': -0.011, 'rd': 0.8608608608608609, 'dij': 2.84, 'Wd': 9.057322241}, 
                  'Ru': {'d_cen': -1.6305, 'rd': 0.7507507507507507, 'dij': 2.72, 'Wd': 8.066324527}, 
                  'Rh': {'d_cen': -2.0874, 'rd': 0.7207207207207207, 'dij': 2.72, 'Wd': 7.39255913}, 
                  'Pd': {'d_cen': -2.087, 'rd': 0.6706706706706707, 'dij': 2.79, 'Wd': 5.679312123}, 
                  'Ag': {'d_cen': -4.1421, 'rd': 0.6606606606606606, 'dij': 2.94, 'Wd': 4.503883909}, 
                  'Ta': {'d_cen': 1.1906, 'rd': 1.021021021021021, 'dij': 2.98, 'Wd': 10.89131422}, 
                  'W': {'d_cen': 0.1732, 'rd': 0.9409409409409409, 'dij': 2.86, 'Wd': 10.73729515}, 
                  'Re': {'d_cen': -1.0633, 'rd': 0.8808808808808809, 'dij': 2.75, 'Wd': 10.99527175}, 
                  'Os': {'d_cen': -1.9693, 'rd': 0.8508508508508509, 'dij': 2.72, 'Wd': 10.63999091}, 
                  'Ir': {'d_cen': -2.6636, 'rd': 0.8208208208208209, 'dij': 2.74, 'Wd': 9.50224736}, 
                  'Pt': {'d_cen': -2.6369, 'rd': 0.7907907907907907, 'dij': 2.82, 'Wd': 7.638177581}, 
                  'Au': {'d_cen': -3.6577, 'rd': 0.7607607607607607, 'dij': 2.95, 'Wd': 5.83120117}}

In [ ]:
unrelax_small_images = Trajectory('Data/clean_images.traj')
copy_images = [image.copy() for image in unrelax_small_images]
unrelax_images = [copy_image.repeat((11,11,1)) for copy_image in copy_images]
tabulated_name = np.loadtxt('Data/Tabulated_names.txt',dtype='str')
tabulated_nbr_index_sorted = np.loadtxt('Data/Tabulated_nbr_index_sorted.txt')[:,:12]
tabulated_d_ij_sorted = np.loadtxt('Data/Tabulated_d_ij_sorted.txt')[:,:12]
tabulated_site_index = np.loadtxt('Data/Tabulated_site_index.txt')
tabulated_site_element = np.loadtxt('Data/Tabulated_site_element.txt',dtype='str')
tabulated_mulliken = np.loadtxt('Data/Tabulated_mulliken.txt')
tabulated_d_cen_inf = np.loadtxt('Data/Tabulated_d_cen_inf.txt')
tabulated_full_width_inf = np.loadtxt('Data/Tabulated_full_width_inf.txt')

parm = np.loadtxt('Data/combined_parm_idx_val_8_idx_test_9.txt')
beta = parm[:,5]
alpha = parm[:,6]
tinnet_Wd = parm[:,4]
dft_Wd = parm[:,3]
tinnet_d_cen = parm[:,2]
dft_d_cen = parm[:,1]
zeta = np.load('Data/combined_zeta_idx_val_8_idx_test_9.npy')[:,:12,0]**(1/7)

### The physics model to calculate d-band center

In [ ]:
def d_center(X):
    m2 = 0
    for i in range(number_nbr):
        V2ds = 7.62**2*3.16**2*reference_data[base_element]['rd']**3/(X[:,3*i+1]*X[:,3*i+2])**7
        V2dd = 7.62**2*(16.2**2+2*8.75**2)*reference_data[base_element]['rd']**3*X[:,3*i+0]**3/(X[:,3*i+1]*X[:,3*i+2])**10
        m2 = m2 + V2ds+V2dd
    result = X[:,-3]*m2**0.5*(ref_ratio-X[:,-2]*X[:,-1])
    return result

### Compute Shapley values

In [ ]:
dij_element_sort = ['Co', 'Ni', 'Cu', 'Os', 'Ru', 'Rh', 'Re', 'Ir', 'Pd', 'Au', 'Ag']

SAA_idx = []
for ele in dij_element_sort:
    for i in range(len(unrelax_images)):
    
        SAA = '111_'+ele+'3M-'+ele+'_Pt'
        if tabulated_name[i] == SAA and tabulated_site_element[i] == 'Pt':
            SAA_idx.append(i)


base_facet = '111'
number_nbr = 12

base_facet_idx = []
for i in range(len(unrelax_images)):
    if tabulated_name[i][:3]==base_facet:
        base_facet_idx.append(i)
base_facet_nbr_element = []
for idx in base_facet_idx:
    base_facet_nbr_element.append([unrelax_images[idx][j].symbol for j in tabulated_nbr_index_sorted[idx].astype(int)])
base_facet_rd_nbr = np.array([[reference_data[base_facet_nbr_element[i][j]]['rd'] for j in range(number_nbr)] for i in range(len(base_facet_idx))])


base_element = 'Pt'

ref_ratio = reference_data[base_element]['d_cen']/reference_data[base_element]['Wd']
base_idx = np.where(tabulated_name==base_facet+'_'+base_element+'_1.000')[0][0]

ele_1 = base_element

selected_delta_cen = []
selected_images = []

selected_rdj = []
selected_dij = []
selected_zeta = []
selected_beta = []
selected_alpha = []
selected_mulliken = []
selected_concen = []

selected_idx = SAA_idx

for kk in range(len(selected_idx)):
    
    idx = selected_idx[kk]
    
    selected_images.append(unrelax_images[idx])
    
    base_element = tabulated_site_element[idx]
    ref_ratio = reference_data[base_element]['d_cen']/reference_data[base_element]['Wd']
    base_idx = np.where(tabulated_name==base_facet+'_'+base_element+'_1.000')[0][0]
    test_idx = np.int64(selected_idx[kk])
    
    selected_delta_cen.append(tinnet_d_cen[test_idx] - tinnet_d_cen[base_idx])

    base_inner_idx = np.where(base_facet_idx==base_idx)[0][0]
    test_inner_idx = np.where(base_facet_idx==test_idx)[0][0]
    base_X_W = np.array([[0.0]*(number_nbr*3+3)])
    for i in range(number_nbr):
        base_X_W[0][3*i+0]=base_facet_rd_nbr[base_inner_idx][i]
        base_X_W[0][3*i+1]=tabulated_d_ij_sorted[base_idx][i]
        base_X_W[0][3*i+2]=zeta[base_idx][i]
    base_X_W[0][-3] = alpha[base_idx]
    base_X_W[0][-2] = beta[base_idx]
    base_X_W[0][-1] = tabulated_mulliken[base_idx]

    test_X_W = np.array([[0.0]*(number_nbr*3+3)])
    for i in range(number_nbr):
        test_X_W[0][3*i+0]=base_facet_rd_nbr[test_inner_idx][i]
        test_X_W[0][3*i+1]=tabulated_d_ij_sorted[test_idx][i]
        test_X_W[0][3*i+2]=zeta[test_idx][i]
    test_X_W[0][-3] = alpha[test_idx]
    test_X_W[0][-2] = beta[test_idx]
    test_X_W[0][-1] = tabulated_mulliken[test_idx]

    explainer = shap.Explainer(d_center, base_X_W)
    shap_values = explainer(test_X_W)
    
    rdj_ = 0
    dij_ = 0
    zeta_ = 0

    alpha_ = shap_values.values[0][-3]
    beta_ = shap_values.values[0][-2]
    mulliken_ = shap_values.values[0][-1]

    selected_alpha.append(alpha_)
    selected_beta.append(beta_)
    selected_mulliken.append(mulliken_)
    
    for i in range(number_nbr):
        rdj_ += shap_values.values[0][3*i+0]
        dij_ += shap_values.values[0][3*i+1]
        zeta_ += shap_values.values[0][3*i+2]

    selected_rdj.append(rdj_)
    selected_dij.append(dij_)
    selected_zeta.append(zeta_)


print('alpha',selected_alpha)


### Plot the correlation between d-band skewness and the contribution from orbital resonance to the d-band center shift

In [ ]:
shap_list = selected_alpha.append(0)
skew_list = [0.8518113496006632,0.8627151914292235,0.6335584979262483,0.5546865255868431,0.7196118767691333,0.6191934681332795,0.9665530507435334,0.3734673168218981,0.47225055591809034,-0.062051725407769494,0.07772478079201128,0.2086482164297922]

symbols = ['s','o','^','v','<','>','+','x','D','d'] # Symbol
lps = [k+'-' for k in ['o','^','v','<','>','s','+','x','D','d']] # Line + Symbol
colors= ['b','r','g','c','m','y','k','w'] # Color
ms = 4
ew = 1.5
rcParams['figure.figsize'] = 1.5*1.5*1.67323,1.5*1.5*1.67323
rcParams['ps.useafm'] = True
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
mpl.rcParams['axes.linewidth'] = 1

rcParams['pdf.fonttype'] = 42
matplotlib.rc('xtick.major', size=6)
matplotlib.rc('xtick.minor', size=3)
matplotlib.rc('ytick.major', size=6)
matplotlib.rc('ytick.minor', size=3)
matplotlib.rc('lines', markeredgewidth=0.5*2)
matplotlib.rc('font', size=8*2.0)

fig, ax = plt.subplots()

X = skew_list
y = shap_list

ax.scatter(X,y,color='green',marker='s',s=50)

ax.minorticks_on()

ax.tick_params(axis="x",direction="in",which='both',)
ax.tick_params(axis="y",direction="in",which='both')

ax.set_xticks([0,0.2,0.4,0.6,0.8,1.0])
ax.set_yticks([-0.6,-0.4,-0.2,0,0.2,0.4])

ax.set_xlabel('Skewness')
ax.set_ylabel('Shapley value (eV)')

Z = np.polyfit(X,y,1)
plt.plot(np.sort(X),(Z[0]*np.array(X)+Z[1])[np.argsort(X)],'--',color='gray',linewidth=1)
corr_matrix = np.corrcoef(X, y)
corr = corr_matrix[0,1]
print('R = '+str(corr))

plt.xlabel('skewness')
plt.ylabel('Shapley value (eV)')

plt.xlim([-0.2,1.1])
plt.ylim([-0.6,0.4])

plt.savefig('Figure1b.png', format='png',transparent = True,dpi=600, bbox_inches='tight')

## Compute Shapley value of each effect contributing to d-band skewness change of SAAs relative to Pt

### Load data

In [ ]:
TB_param = pickle.load(open('Data/TB_SAA.pkl','rb'))
sort_index_base = TB_param[0][15][:49]

atoms_ref = io.read('Data/atoms_ref_TinNet.traj')

X_base = np.array([list(pd.read_csv('Data/shap_test_12_TinNet_reduce.csv')['X_base'])])
X_test = np.array([list(pd.read_csv('Data/shap_test_12_TinNet_reduce.csv')['X_test_'+str(iii)]) for iii in range(12)])

### Define the function to calculate d-band skewness from parameterized Hamiltonian matrix

In [ ]:
def NMM3(X):
    result = np.array([0.0]*len(X))
    for ii in range(len(X)):
        Natm = 49
        HSH = np.ones((Natm,Natm))
        count = 0
        for mm in range(Natm):
            for nn in range(Natm):
                if mm < nn:
                    HSH[mm][nn] = X[ii,296:][count]
                    HSH[nn][mm] = HSH[mm][nn]
                    count += 1
        #HSH = X[ii,296:].reshape(49,49)
        OSH = X[ii,2:296]
        

        all_dij = atoms_ref.get_all_distances()
        nbr_dis = atoms_ref.get_all_distances()[975]
        all_R = atoms_ref.get_positions()[:,None,:] - atoms_ref.get_positions()[None,:,:]
        sort_index = sort_index_base
        sort_element = np.array(atoms_ref.get_chemical_symbols())[sort_index]
        sort_rd = [reference_data[ele]['rd'] for ele in sort_element]
        
        host_idx_list = []

        for i in range(len(sort_element)):
            if sort_element[i] == 'Ag':
                host_idx_list.append(i)

        all_dij = X[ii,0]/2.94*all_dij
        nbr_dis = X[ii,0]/2.94*nbr_dis
        all_R = X[ii,0]/2.94*all_R


        for jj in host_idx_list:
            sort_rd[jj] = X[ii,1]


        Natm = 49
        H = np.zeros((Natm*6,Natm*6))

        orb_list = ['dxy','dyz','dxz','dz2','dx2-z2','s']

        for i in range(Natm):
            for j in range(Natm):

                if i != j:
                    rd_1 = sort_rd[i]
                    rd_2 = sort_rd[j]
                    dij = all_dij[sort_index[i]][sort_index[j]]
                    r = all_R[sort_index[i]][sort_index[j]]
                    l, m, n = r/np.linalg.norm(r)

                    if dij <= 3.6:
                        
                        dij = dij*HSH[i][j]


                        v_ss_sigma = 7.62*(-1.4)*(1/dij**2)
                        v_sd_sigma = 7.62*(-3.16)*(rd_2**1.5/dij**3.5)
                        v_ds_sigma = 7.62*(-3.16)*(rd_1**1.5/dij**3.5)

                        v_dd_sigma = 7.62*(-16.2)*(rd_1**1.5*rd_2**1.5/dij**5)
                        v_dd_pi = 7.62*(8.75)*(rd_1**1.5*rd_2**1.5/dij**5)
                        v_dd_delta = 0

                        H[6*i+5][6*j+5] = v_ss_sigma

                        H[6*i+5][6*j+1] = 3**0.5*l*m*v_sd_sigma
                        H[6*i+5][6*j+2] = 3**0.5*m*n*v_sd_sigma
                        H[6*i+5][6*j+3] = 3**0.5*l*n*v_sd_sigma
                        H[6*i+5][6*j+4] = (n**2 - (l**2 + m**2)/2)*v_sd_sigma
                        H[6*i+5][6*j+0] = 3**0.5/2*(l**2-m**2)*v_sd_sigma

                        H[6*i+1][6*j+5] = 3**0.5*l*m*v_ds_sigma
                        H[6*i+2][6*j+5] = 3**0.5*m*n*v_ds_sigma
                        H[6*i+3][6*j+5] = 3**0.5*l*n*v_ds_sigma
                        H[6*i+4][6*j+5] = (n**2 - (l**2 + m**2)/2)*v_ds_sigma
                        H[6*i+0][6*j+5] = 3**0.5/2*(l**2-m**2)*v_ds_sigma

                        H[6*i+1][6*j+1] = 3*l**2*m**2*v_dd_sigma + (l**2 + m**2 - 4*l**2*m**2)*v_dd_pi + (n**2 + l**2*m**2)*v_dd_delta
                        H[6*i+1][6*j+2] = H[6*i+2][6*j+1]  = l*n*(3*m**2*v_dd_sigma + (1 - 4*m**2)*v_dd_pi + (m**2 - 1)*v_dd_delta)
                        H[6*i+1][6*j+3] = H[6*i+3][6*j+1] = m*n*(3*l**2*v_dd_sigma + (1 - 4*l**2)*v_dd_pi + (l**2 - 1)*v_dd_delta)
                        H[6*i+1][6*j+4] = H[6*i+4][6*j+1] = 3**0.5*l*m*((n**2 - 0.5*(l**2 + m**2))*v_dd_sigma - 2*n**2*v_dd_pi + 0.5*(1 + n**2)*v_dd_delta)
                        H[6*i+1][6*j+0] = H[6*i+0][6*j+1] = l*m*(l**2 - m**2)*(1.5*v_dd_sigma - 2*v_dd_pi + 0.5*v_dd_delta)

                        H[6*i+2][6*j+2] = 3*n**2*m**2*v_dd_sigma + (n**2 + m**2 - 4*n**2*m**2)*v_dd_pi + (l**2 + n**2*m**2)*v_dd_delta
                        H[6*i+2][6*j+3] = H[6*i+3][6*j+2] = m*l*(3*n**2*v_dd_sigma + (1 - 4*n**2)*v_dd_pi + (n**2 - 1)*v_dd_delta)
                        H[6*i+2][6*j+4] = H[6*i+4][6*j+2] = 3**0.5*m*n*((n**2 - 0.5*(l**2 + m**2))*v_dd_sigma + (l**2 + m**2 - n**2)*v_dd_pi - 0.5*(l**2 + m**2)*v_dd_delta)
                        H[6*i+2][6*j+0] = H[6*i+0][6*j+2] = m*n*(1.5*(l**2 - m**2)*v_dd_sigma - (1 + 2*(l**2 - m**2))*v_dd_pi + (1 + 0.5*(l**2 - m**2))*v_dd_delta)

                        H[6*i+3][6*j+3] = 3*l**2*n**2*v_dd_sigma + (l**2 + n**2 - 4*l**2*n**2)*v_dd_pi + (m**2 + l**2*n**2)*v_dd_delta
                        H[6*i+3][6*j+4] = H[6*i+4][6*j+3] = 3**0.5*l*n*((n**2 - 0.5*(l**2 + m**2))*v_dd_sigma + (l**2 + m**2 - n**2)*v_dd_pi - 0.5*(l**2 + m**2)*v_dd_delta)
                        H[6*i+3][6*j+0] = H[6*i+0][6*j+3] = n*l*(1.5*(l**2 - m**2)*v_dd_sigma + (1 - 2*(l**2 - m**2))*v_dd_pi - (1 - 0.5*(l**2 - m**2))*v_dd_delta)

                        H[6*i+4][6*j+4] = (n**2 - 0.5*(l**2 + m**2))**2*v_dd_sigma + 3*n**2*(l**2 + m**2)*v_dd_pi + 0.75*(l**2 + m**2)**2*v_dd_delta
                        H[6*i+4][6*j+0] = H[6*i+0][6*j+4] = 3**0.5*(l**2 - m**2)*(0.5*(n**2 - 0.5*(l**2 + m**2))*v_dd_sigma - n**2*v_dd_pi + 0.25*(1 + n**2)*v_dd_delta)

                        H[6*i+0][6*j+0] = 0.75*(l**2 - m**2)**2*v_dd_sigma + (l**2 + m**2 - (l**2 - m**2)**2)*v_dd_pi + (n**2 + 0.25*(l**2 - m**2)**2)*v_dd_delta

                if i == j:
                    H[6*i+5][6*j+5] = OSH[6*i+0]
                    H[6*i+1][6*j+1] = OSH[6*i+1]
                    H[6*i+2][6*j+2] = OSH[6*i+2]
                    H[6*i+3][6*j+3] = OSH[6*i+3]
                    H[6*i+4][6*j+4] = OSH[6*i+4]
                    H[6*i+0][6*j+0] = OSH[6*i+5]
        HHH = H@H@H
        HH = H@H
        m3 = 0
        for i in range(5):
            m3 = m3 + HHH[i][i]

        m2 = 0
        for i in range(5):
            m2 = m2 + HH[i][i]

        norm_m3 = m3/m2**1.5
        result[ii] = norm_m3
    return result

### Compuate Shapley values

In [ ]:
explainer = shap.Explainer(NMM3, X_base)
shap_values_base = explainer(X_base,max_evals=1027)
shap_values_test = explainer(X_test,max_evals=1027)


shap_test_dic = {'base':shap_values_base.values[0],'test_0':shap_values_test.values[0],'test_1':shap_values_test.values[1],'test_2':shap_values_test.values[2],'test_3':shap_values_test.values[3],'test_4':shap_values_test.values[4],'test_5':shap_values_test.values[5],'test_6':shap_values_test.values[6],'test_7':shap_values_test.values[7],'test_8':shap_values_test.values[8],'test_9':shap_values_test.values[9],'test_10':shap_values_test.values[10],'test_11':shap_values_test.values[11]}
shap_test_df = pd.DataFrame(shap_test_dic)
shap_test_df.to_csv('Data/shap_values_12_TinNet_M3.csv')

## Visualize the correlation between the physical quantities and Shapley values of orbital resonance and orbital coupling effects contributing to d-band skewness change 

### Prepare data

In [11]:
shap_data = pd.read_csv('Data/shap_values_12_TinNet_M3.csv')
sum_d_list = []

for i in range(12):
    shap = list(shap_data['test_'+str(i)])[2:296]
    sum_d = 0
    for j in range(49):
        sum_d += shap[6*j+1]
        sum_d += shap[6*j+2]
        sum_d += shap[6*j+3]
        sum_d += shap[6*j+4]
        sum_d += shap[6*j+5]
    sum_d_list.append(sum_d)

sum_dij_list = []
for i in range(12):
    shap = [list(shap_data['test_'+str(i)])[0]]
    sum_dij = sum(shap)
    sum_dij_list.append(sum_dij)
    
sum_rd_list = []
for i in range(12):
    shap = [list(shap_data['test_'+str(i)])[1]]
    sum_rd = sum(shap)
    sum_rd_list.append(sum_rd)

element_list = ['Ag','Au','Ir','Cu','Pt','Pd','Rh','Os','Ni','Ru','Co','Re']
d_levels = np.array([reference_data[ele]['d_cen'] for ele in element_list])

### Plot for orbital resonance effect

In [ ]:
symbols = ['s','o','^','v','<','>','+','x','D','d'] # Symbol
lps = [k+'-' for k in ['o','^','v','<','>','s','+','x','D','d']] # Line + Symbol
colors= ['b','r','g','c','m','y','k','w'] # Color
ms = 4
ew = 1.5
rcParams['figure.figsize'] = 1.5*1.5*1.67323,1.5*1.5*1.67323
rcParams['ps.useafm'] = True
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
mpl.rcParams['axes.linewidth'] = 1

rcParams['pdf.fonttype'] = 42
matplotlib.rc('xtick.major', size=6)
matplotlib.rc('xtick.minor', size=3)
matplotlib.rc('ytick.major', size=6)
matplotlib.rc('ytick.minor', size=3)
matplotlib.rc('lines', markeredgewidth=0.5*2)
matplotlib.rc('font', size=8*2.0)


fig, ax = plt.subplots()

X = d_levels - d_levels[4]
y = sum_d_list

plt.scatter(X,y,s=50,marker='s',color = 'orange')

ax.set_xlabel(r'$d$-level misalignment (eV)')
ax.set_ylabel('Shapley value')

plt.xlim([-5- d_levels[4],-0.5- d_levels[4]])
plt.ylim([-0.6,0.65])

plt.minorticks_on()
plt.tick_params(axis="y",direction="in",which='both')
plt.tick_params(axis="x",direction="in",which='both')

plt.xticks([-2,-1,0,1,2])
plt.yticks([-0.5,-0.25,0,0.25,0.5])

Z = np.polyfit(X,y,1)
corr_matrix = np.corrcoef(X, y)
corr = corr_matrix[0,1]
print('R = '+str(corr))
X_fit = np.linspace(X.min(),X.max(),10)

plt.plot(np.sort(X_fit),(Z[0]*X_fit+Z[1])[np.argsort(X_fit)],'--',color='gray',linewidth=1)


#plt.savefig('Figure1c.png', format='png',transparent = True,dpi=600, bbox_inches='tight')

### Plot for orbital coupling effect

In [ ]:
symbols = ['s','o','^','v','<','>','+','x','D','d'] # Symbol
lps = [k+'-' for k in ['o','^','v','<','>','s','+','x','D','d']] # Line + Symbol
colors= ['b','r','g','c','m','y','k','w'] # Color
ms = 4
ew = 1.5
rcParams['figure.figsize'] = 1.5*1.5*1.67323,1.5*1.5*1.67323
rcParams['ps.useafm'] = True
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
mpl.rcParams['axes.linewidth'] = 1

rcParams['pdf.fonttype'] = 42
matplotlib.rc('xtick.major', size=6)
matplotlib.rc('xtick.minor', size=3)
matplotlib.rc('ytick.major', size=6)
matplotlib.rc('ytick.minor', size=3)
matplotlib.rc('lines', markeredgewidth=0.5*2)
matplotlib.rc('font', size=8*2.0)


fig, ax = plt.subplots()

X = 762*np.array([reference_data[ele]['rd'] for ele in element_list])**1.5/np.array([reference_data[ele]['dij']**5 for ele in element_list])
y = np.array(sum_rd_list) + np.array(sum_dij_list)

plt.scatter(X,y,s=50,marker='s',color = 'brown')

ax.set_xlabel('TB coupling strength (a.u.)')
ax.set_ylabel('Shapley value')

plt.xlim([1.5,4.5])
plt.ylim([-0.5,0.8])

plt.minorticks_on()
plt.tick_params(axis="y",direction="in",which='both')
plt.tick_params(axis="x",direction="in",which='both')

plt.xticks([])

Z = np.polyfit(X,y,1)
corr_matrix = np.corrcoef(X, y)
corr = corr_matrix[0,1]
print('R = '+str(corr))
X_fit = np.linspace(X.min(),X.max(),10)

plt.plot(np.sort(X_fit),(Z[0]*X_fit+Z[1])[np.argsort(X_fit)],'--',color='gray',linewidth=1)


#plt.savefig('Figure1d.png', format='png',transparent = True,dpi=600, bbox_inches='tight')